<a href="https://colab.research.google.com/github/anassdrs/plant_identification/blob/main/plants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd  # data processing
import os #  to interact with files using there paths
import cv2
from sklearn.datasets import load_files
import torch
from torchvision import transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import time
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# # Unzip the uploaded data into google drive 
#!unzip "/content/drive/MyDrive/les_photos_de_validation_ayoub" -d "/content/drive/MyDrive"

In [ ]:
# Path to a directory with image dataset and subfolders for training, validation and final testing
DATA_PATH = r"drive/MyDrive"

# Number of threads for data loader 
NUM_WORKERS = 4  

# Image size: even though image sizes are bigger than 64, we use this to speed up training
SIZE_H = SIZE_W = 350

# Number of classes in the dataset
NUM_CLASSES = len(sorted(os.listdir(os.path.join(DATA_PATH + '/train'))))
print(NUM_CLASSES)
# Epochs: number of passes over the training data, we use it this small to reduce training babysitting time
EPOCH_NUM = 10

# Batch size: for batch gradient descent optimization, usually selected as 2**K elements
BATCH_SIZE = 16

# Images mean and std channelwise
image_mean = [0, 0, 0]
image_std  = [1, 1, 1]

# # Last layer (embeddings) size for CNN models
# EMBEDDING_SIZE = 128

In [ ]:
from tqdm import tqdm
from matplotlib import pyplot as plt
DATA_PATH="drive/MyDrive"
plt.figure(figsize=(15,10))

i = 1
for stage in ['train', 'val', 'les_photos_de_validation']:
    count_samples_for_class = dict()
    for directory in tqdm(sorted(os.listdir(os.path.join(DATA_PATH + '/' + stage)))):
        count_samples_for_class[directory] = len(os.listdir(os.path.join(DATA_PATH + '/' + stage + '/' + directory)))
    plt.subplot(2,2,i)
    plt.bar(count_samples_for_class.keys(), count_samples_for_class.values(), width=20, color='g', label=stage)
    plt.legend()
    i+=1
plt.show()

In [ ]:
transformer = transforms.Compose([
    transforms.Resize((350, 350)),       # scaling images to fixed size
    transforms.ToTensor(),                      # converting to tensors
    transforms.Normalize(image_mean, image_std) # normalize image data per-channel
])

In [ ]:
# load dataset using torchvision.datasets.ImageFolder
train_dataset = torchvision.datasets.ImageFolder(os.path.join(DATA_PATH, 'train'), transform=transformer)
val_dataset = torchvision.datasets.ImageFolder(os.path.join(DATA_PATH, 'val'), transform=transformer)
# load test data also, to be used for final evaluation
test_dataset = torchvision.datasets.ImageFolder(os.path.join(DATA_PATH, 'les_photos_de_validation'), transform=transformer)

In [ ]:
n_train, n_val, n_test = len(train_dataset), len(val_dataset), len(test_dataset)
n_train, n_val, n_test

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              num_workers=NUM_WORKERS)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            num_workers=NUM_WORKERS)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=BATCH_SIZE,
                                             num_workers=NUM_WORKERS)
classes = train_dataset.classes
print(classes)
print(type(classes))
import json


with open('drive/MyDrive/classes.json', 'w') as jsonfile:
    json.dump(classes, jsonfile)

#encoder and decoder to convert classes into integer
decoder = {}
for i in range(len(classes)):
    decoder[classes[i]] = i
encoder = {}
for i in range(len(classes)):
    encoder[i] = classes[i]
print(type(train_loader))
x,y=next(iter(train_loader))
print(len(y))

In [ ]:
def plot_from_batch_generator(batch_gen):
    data_batch, label_batch = next(iter(batch_gen))
    grid_size = (3, 3)
    f, axarr = plt.subplots(*grid_size)
    f.set_size_inches(30,20)
    class_names = batch_gen.dataset.classes
    for i in range(grid_size[0] * grid_size[1]):
        # read images from batch to numpy.ndarray and change axes order [H, W, C] -> [H, W, C]
        batch_image_ndarray = np.transpose(data_batch[i].numpy(), [1, 2, 0])
        # inverse normalization for image data values back to [0,1] and clipping the values for correct pyplot.imshow()
        src = np.clip(image_std * batch_image_ndarray + image_mean, 0, 1)
        
        # display batch samples with labels
        sample_title = '(%s)' % ( class_names[label_batch[i]])
        axarr[i // grid_size[0], i % grid_size[0]].imshow(src)
        axarr[i // grid_size[0], i % grid_size[0]].set_title(sample_title)
    pass

In [ ]:
plot_from_batch_generator(train_loader)


In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from keras.utils.vis_utils import plot_model
import tensorflow as tf
IMG_SHAPE = (350, 350, 3) 
model0 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
tf.keras.utils.plot_model(model0) # pour dessiner et visualiser 
a=model0.summary() # pour voir la liste des couches et des paramètres
plot_model(model0, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
import os
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
from efficientnet_pytorch import EfficientNet
from collections import OrderedDict


model = EfficientNet.from_pretrained('efficientnet-b4')

# Disable gradient updates for all the layers except the final layer
for p in model.parameters():
    p.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_feat = model._fc.in_features
model._fc = nn.Linear(num_feat, NUM_CLASSES)

# Use available device for calculations
model = model.to(device)

In [ ]:
from IPython.display import clear_output
import time
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from torch.optim import lr_scheduler


def fit(model, augmentation, shedul, epochs, lr):
    torch.cuda.empty_cache()
    
    opt = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))
    
    if shedul==1:
        scheduler = lr_scheduler.StepLR(opt, step_size=1, gamma=0.8)
    
    
    train_losses = []
    val_losses = []
    train_accuracy = []
    val_accuracy = []
    start = time.time()
    a_train=[]
    a_val=[]
    l_train=[]
    l_val=[]
    for epoch in range(epochs):
        correct = []
        loss1=[]
        model.train(True)
        for img, label in train_loader:
            clear_output(True)
            img = img.to(device)
            label = label.to(device)
            opt.zero_grad()
            preds = model(img).to(device)
            ce_loss = nn.CrossEntropyLoss()
            loss = ce_loss(preds, label)
            loss.backward()
            opt.step()
            tmp_loss = loss.item()
            train_losses.append(tmp_loss)

            end = time.time() - start
            if shedul==1:
                print(f'epoch = {epoch}, Learning rate =', scheduler.get_lr())

            acc = accuracy_score(label.cpu(), np.argmax(preds.cpu().detach().numpy(), axis=1))
            train_accuracy.append(acc)
            correct.append(acc)
            loss1.append(tmp_loss)
            plt.figure(figsize=(15,13))
            plt.subplot(2,1,1)
            plt.plot(np.arange(0, len(train_losses)), train_losses, label=f'spent_time={end}, ep={epoch}')
            plt.legend()

            plt.subplot(2,1,2)
            plt.plot(np.arange(0, len(train_accuracy)), train_accuracy, label='train_accuracy')
            plt.legend()
            plt.show()
        accuracy = sum(correct)/len(correct)
        l= sum(loss1)/len(loss1)
        # trainset, not train_loader
        # probably x in your case
        a_train.append(accuracy)
        l_train.append(l)
        print("Accuracy = {}".format(a_train))
        if augmentation == 1:
            for img, label in train_loader_aug:
                clear_output(True)
                img = img.to(device)
                label = label.to(device)
                opt.zero_grad()
                preds = model(img).to(device)
                ce_loss = nn.CrossEntropyLoss()
                loss = ce_loss(preds, label)
                loss.backward()
                opt.step()
                tmp_loss = loss.item()
                train_losses.append(tmp_loss)

                end = time.time() - start

                acc = accuracy_score(label.cpu(), np.argmax(preds.cpu().detach().numpy(), axis=1))
                train_accuracy.append(acc)
              
                plt.figure(figsize=(15,13))
                plt.subplot(2,1,1)
                plt.plot(np.arange(0, len(train_losses)), train_losses, label=f'aug| spent_time={end}, ep={epoch}')
                plt.legend()

                plt.subplot(2,1,2)
                plt.plot(np.arange(0, len(train_accuracy)), train_accuracy, label='aug| train_accuracy')
                plt.legend()
                plt.show()
                
        if shedul==1:
            scheduler.step()
        model.train(False)
        with torch.no_grad():
            correct = []
            loss1=[]
            for img, label in tqdm(val_loader):
                clear_output(True)
                img = img.to(device)
                label = label.to(device)
                
                preds = model(img).to(device)
                ce_loss = nn.CrossEntropyLoss()
                loss = ce_loss(preds, label)
                tmp_loss = loss.item()
                val_losses.append(tmp_loss)

                acc = accuracy_score(label.cpu(), np.argmax(preds.cpu().detach().numpy(), axis=1))
                val_accuracy.append(acc)
                correct.append(acc)
                loss1.append(tmp_loss)
                plt.figure(figsize=(15,13))
                plt.subplot(2,1,1)
                plt.plot(np.arange(0, len(val_losses)), val_losses, label=f'ep={epoch}')
                plt.legend()

                plt.subplot(2,1,2)
                plt.plot(np.arange(0, len(val_accuracy)), val_accuracy, label='val_accuracy')
                plt.legend()
                plt.show()
            accuracy = sum(correct)/len(correct)
            l=sum(loss1)/len(loss1)
            # trainset, not train_loader
            # probably x in your case
            a_val.append(accuracy)
            l_val.append(l)
            print("Accuracy = {}".format(a_val))
    return model, train_losses, train_accuracy, val_losses, val_accuracy, a_train,a_val,l_train,l_val

In [ ]:
EPOCH_NUM=30
augmentation=0
shedul=0
model1, train_losses, train_accuracy, val_losses, val_accuracy ,a_train,a_val,l_train,l_val= fit(model, augmentation, shedul, EPOCH_NUM , lr=0.003)

In [ ]:
print(a_train)
print(a_val)

In [ ]:
accuracy_train=[0.3960703822307011, 0.5233997070245964, 0.5459945833617225, 0.5497334264495469, 0.5591955099816038, 0.5725114124139811, 0.577344654902228, 0.5774936976221299, 0.5751728895550862, 0.5833787558765415, 0.5837194249506029, 0.5823397152006541, 0.5916272058322545, 0.5851076514274034, 0.592040267084554, 0.589727975744362, 0.5940587313483683, 0.5929515568576685, 0.5914440962049465, 0.5921637596239013, 0.593603086461811, 0.595238298017306, 0.5944760509640935, 0.5925257205150917, 0.5937563875451387, 0.5938372964502283, 0.6007188117462696, 0.5929685903113715, 0.6000246985078694, 0.6030183279961845]
accuracy_val=[0.5469858156028369, 0.5877659574468085, 0.5842198581560284, 0.5966312056737588, 0.5886524822695035, 0.5953014184397163, 0.5970744680851063, 0.600177304964539, 0.6006205673758865, 0.6032801418439716, 0.6063829787234043, 0.6085992907801419, 0.6112588652482269, 0.613031914893617, 0.612145390070922, 0.6139184397163121, 0.6236702127659575, 0.6099290780141844, 0.6117021276595744, 0.6054964539007093, 0.6134751773049646, 0.6117021276595744, 0.612145390070922, 0.6187943262411347, 0.6148049645390071, 0.6205673758865248, 0.6148049645390071, 0.6125886524822695, 0.6108156028368794, 0.6174645390070922]
epochs = range(0,30)
print(max(accuracy_val))
plt.plot(epochs, accuracy_train, 'b', label='accuracy_train')
plt.plot(epochs, accuracy_val, 'y', label='accuracy_val')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
def plot_results(train_losses, train_accuracy, val_losses, val_accuracy,a_train,a_val,l_train,l_val):
    plt.figure(figsize=(25,21))
    plt.subplot(2,2,1)
    plt.plot(np.arange(0, len(l_train)), l_train, label=f'train_loss')
    plt.legend()
    plt.subplot(2,2,3)
    plt.plot(np.arange(0, len(l_val)), l_val, label=f'val_loss')
    plt.legend()
    plt.subplot(2,2,2)
    plt.plot(np.arange(0, len(a_train)), a_train, label='train_accuracy')
    plt.legend()
    plt.subplot(2,2,4)
    plt.plot(np.arange(0, len(a_val)), a_val, label='val_accuracy')
    plt.legend()
    plt.show()
    print(train_accuracy)
    print(a)
    

In [ ]:
plot_results(train_losses, train_accuracy, val_losses, val_accuracy,a_train,a_val,l_train,l_val)

In [ ]:
#Predict
from PIL import Image
import numpy as np
import cv2
def prf(model):
  val_losses = []

  val_accuracy = []
  start = time.time()

  a_val=[]

  l_val=[]
  with torch.no_grad():
            correct = []
            loss1=[]
            for img, label in tqdm(val_loader):
                clear_output(True)
                img = img.to(device)
                label = label.to(device)
                preds = model(img).to(device)
                print(label)
                print(preds.cpu().detach().numpy())
                ce_loss = nn.CrossEntropyLoss()
                loss = ce_loss(preds, label)
                tmp_loss = loss.item()
                val_losses.append(tmp_loss)

                acc = accuracy_score(label.cpu(), np.argmax(preds.cpu().detach().numpy(), axis=1))
                val_accuracy.append(acc)
                correct.append(acc)
                loss1.append(tmp_loss)
                plt.figure(figsize=(15,13))
                plt.subplot(2,1,1)
                plt.plot(np.arange(0, len(val_losses)), val_losses, label=f'ep={1}')
                plt.legend()

                plt.subplot(2,1,2)
                plt.plot(np.arange(0, len(val_accuracy)), val_accuracy, label='val_accuracy')
                plt.legend()
                plt.show()
            accuracy = sum(correct)/len(correct)
            l=sum(loss1)/len(loss1)
            # trainset, not train_loader
            # probably x in your case
            a_val.append(accuracy)
            l_val.append(l)
            print("Accuracy = {}".format(a_val))

def predict(model,image,device,encoder,transforms = None):
    #model = torch.load('./model.h5')
    model.eval()
    if(isinstance(image,np.ndarray)):
      image = Image.fromarray(image)
    if(transforms!=None):
        image = transforms(image)
    data = image.expand(1,-1,-1,-1)
    data = data.type(torch.FloatTensor).to(device)
    sm = nn.Softmax(dim = 1)
    output = model(data)
    output = sm(output)
    _, preds = torch.max(output, 1)
    #img_plot(image)
    result=prediction_bar(output,encoder)
    return preds,result
def prediction_bar(output,encoder):
    output = output.cpu().detach().numpy()
    a = output.argsort()
    a = a[0]
    
    size = len(a)
    if(size>5):
        a = np.flip(a[-5:])
    else:
        a = np.flip(a[-1*size:])
    prediction = list()
    clas = list()
    for i in a:
      prediction.append(float(output[:,i]*100))
      clas.append(str(i))
    for i in a:
        print('Class: {} , confidence: {}'.format(encoder[int(i)],float(output[:,i]*100)))
    #plt.bar(clas,prediction)
    #plt.title("Confidence score bar graph")
    #plt.xlabel("Confidence score")
    #plt.ylabel("Class number")
    print('_____________________________________')
    print(encoder[int(clas[0])])
    return encoder[int(clas[0])]
def img_plot(image,inv_normalize = None):
    if(inv_normalize!=None):
        image = inv_normalize(image)
    image = image.cpu().numpy().transpose(1,2,0)
    plt.imshow(image)
    plt.show()
image = cv2.imread('/content/drive/MyDrive/les_photos_de_validation/09022_Plantae_Tracheophyta_Magnoliopsida_Myrtales_Myrtaceae_Myrtus_communis/52e9135e-4c29-4847-861c-cb1612e179f9.jpg')
mo = torch.load('/content/drive/MyDrive/model_torch')
pred = predict(mo,image,device,encoder,transformer)
#prf(mo)
true={}
false={}
for i in sorted(os.listdir(os.path.join(DATA_PATH + '/les_photos_de_validation'))):
  true[i]=0
  for a in sorted(os.listdir(os.path.join(DATA_PATH + '/les_photos_de_validation/'+i))):
    image = cv2.imread(os.path.join(DATA_PATH + '/les_photos_de_validation/'+i+'/'+a))
    pred,res = predict(mo,image,device,encoder,transformer)
    print(i)
    if(res==i):
      true[i]=true[i]+1
      print("true")
print(true['06115_Plantae_Tracheophyta_Liliopsida_Commelinales_Commelinaceae_Tradescantia_zebrina'])

In [ ]:
s=0
for i in sorted(os.listdir(os.path.join(DATA_PATH + '/les_photos_de_validation'))):
  print(true[i])
  s=s+true[i]
print(s)

In [ ]:
PATH = '/content/drive/MyDrive/model_torch'
torch.save(model1, PATH)


In [ ]:
testdataset = torchvision.datasets.ImageFolder(os.path.join(DATA_PATH, 'les_photos_de_validation'), transform=transformer)

In [ ]:
testloader = torch.utils.data.DataLoader(testdataset,
                                             batch_size=BATCH_SIZE,
                                             num_workers=NUM_WORKERS)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
classes = os.listdir('drive/MyDrive//train') # Get names of classes
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
net = model1
net.load_state_dict(torch.load(PATH))

In [ ]:
# Validation function.
def validate(model1, testloader, criterion):
    model1.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1
            
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
        
    # Loss and accuracy for the complete epoch.
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net.predict(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
train_transform = transforms.Compose([
     transforms.Resize((SIZE_H, SIZE_W)),
     transforms.RandomInvert(),
     transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
     transforms.RandomAdjustSharpness(sharpness_factor=2),
     transforms.RandomSolarize(threshold=192.0),
     transforms.RandomEqualize(),
     transforms.ToTensor(),
     transforms.Normalize(image_mean, image_std),
     ])
train_dataset_aug = torchvision.datasets.ImageFolder(os.path.join(DATA_PATH, 'train'), transform=train_transform)

In [ ]:
len(train_dataset_aug)

In [ ]:
train_loader_aug = torch.utils.data.DataLoader(train_dataset_aug, 
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              num_workers=NUM_WORKERS)

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b4')

# Disable gradient updates for all the layers except the final layer
for p in model.parameters():
    p.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_feat = model._fc.in_features
model._fc = nn.Linear(num_feat, NUM_CLASSES)

# Use available device for calculations
model = model.to(device)

In [ ]:
EPOCH_NUM=20
augmentation=1
shedul=0
model2, train_losses2, train_accuracy2, val_losses2, val_accuracy2 = fit(model, augmentation, shedul, EPOCH_NUM , lr=0.0003)

In [ ]:
plot_results(train_losses2, train_accuracy2, val_losses2, val_accuracy2)

In [ ]:
# create a function to save a model
import datetime
def save_model(model):
  """
  saves a given model in a models directory and appends a suffix (string).
  """
  # Create a model directory pathname with current time
  modeldir = os.path.join("/content/drive/MyDrive/model",
                          datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
  model_path = modeldir + "-" +  ".h5" # save format of model
  print(f"saving model to : {model_path}...")
  model.save(model_path)
  return model_path


In [ ]:
torch.save(model1, '/content/drive/MyDrive/model.h5')

In [ ]:
def edge_and_cut(img):
    emb_img = img.copy()
    edges = cv2.Canny(img, 200, 300)
    
    return edges
filepath = "drive/MyDrive/folder_image_Morocco_training_image_plants/05791_Plantae_Tracheophyta_Liliopsida_Alismatales_Araceae_Alocasia_odora/0cc30b70-03dc-409a-9488-cdee3e6124d5.jpg"
image = cv2.imread(filepath) 
img=edge_and_cut(image)
print(img.shape)



In [ ]:
Xtrain=[]
Xtest=[]
ytrain=[]
ytest=[]
Xtest=[]
Xvalid=[]
ytest=[]
yvalid=[]

def label_img(classes_name,category):
    arr = np.zeros((len(classes_name),), dtype = int)
    arr[classes.index(category), ] = 1
    return arr
classes_num = 85
def create_data(classes_name,l,folder,X,y):
    for category in classes_name:
        imgs = 0
        category_path = os.path.join(folder, category)
        label = label_img(classes_name,category)
        for img in os.listdir(category_path):
            imgs = imgs + 1
            if imgs == l:
                break
            frame = cv2.imread(os.path.join(category_path, img))
            #print(frame.shape)
            img = cv2.resize(frame, ( 150,150))
            #img_array=edge_and_cut(img)
            img_array=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            print(img_array.shape)
            X.append(img_array/255.0)
            y.append(label)
data_dir =  r'drive/MyDrive/train'


classes = os.listdir(data_dir) # Get names of classes
len(classes)
create_data(classes,100,data_dir,Xtrain,ytrain)

In [ ]:
data_dir =  r'drive/MyDrive/test'


classes = os.listdir(data_dir) # Get names of classes
len(classes)
create_data(classes,30,data_dir,Xtest,ytest)

In [ ]:
data_dir =  r'drive/MyDrive/val'


classes = os.listdir(data_dir) # Get names of classes
len(classes)
create_data(classes,30,data_dir,Xvalid,yvalid)

In [ ]:
import cv2 
import matplotlib.pyplot as plt
filepath = "drive/MyDrive/folder_image_Morocco_training_image_plants/05791_Plantae_Tracheophyta_Liliopsida_Alismatales_Araceae_Alocasia_odora/0cc30b70-03dc-409a-9488-cdee3e6124d5.jpg"
image = cv2.imread(filepath) 
print(type(image))
height, width = image.shape[:2] 
  
print("The height of the image is: ", height) 
print("The width of the image is: ", width) 



    




In [ ]:
Xtrain=np.asarray(Xtrain)
ytrain=np.asarray(ytrain)
Xtest=np.asarray(Xtest)
ytest=np.asarray(ytest)
Xvalid=np.asarray(Xvalid)
yvalid=np.asarray(yvalid)
print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)
print(Xvalid.shape)
print(yvalid.shape)
#train = np.array(train)
#shuffle(train)
print(Xtrain.shape[1:])


In [ ]:
import tensorflow as tf

import tensorflow.keras.layers as L

from tensorflow.keras.models import Model

from tensorflow.keras.applications import DenseNet121
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [ ]:
#TL pecific modules
from keras.applications.vgg16 import VGG16
import time
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers as Optimizer
weights_path='/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model=VGG16(include_top=False, weights=None,input_shape=(150,150,3), pooling='avg')
base_model.load_weights(weights_path)
base_model.summary()
model=Sequential()
model.add(base_model)

model.add(Dense(256,activation='relu'))
model.add(Dense(85,activation='softmax'))
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(Xtrain)
epochs=150
batch_size=32
red_lr=ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=2, verbose=1)
model.summary()
base_model.trainable=False # setting the VGG model to be untrainable.
model.compile(optimizer=Adam(lr=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
History = model.fit_generator(datagen.flow(Xtrain,ytrain, batch_size=batch_size),
                              epochs = 150, validation_data = (Xtest,ytest),
                              verbose = 1, steps_per_epoch=Xtrain.shape[0] // batch_size)

In [ ]:
model = tf.keras.Sequential([tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(350, 350,3),
                                             weights='imagenet',
                                             include_top=False),
                                             L.GlobalAveragePooling2D(),
                                             L.Dense(85,
                                             activation='softmax')])
        
model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers as Optimizer
model.compile( optimizer=Optimizer.Adam(lr=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Time to train our model !
epochs = 100#100
batch_size=64#32
 
train_datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=True
)
 
test_datagen = ImageDataGenerator()
 
train_generator = train_datagen.flow(
    Xtrain,ytrain,
    batch_size=batch_size)
 
validation_generator = test_datagen.flow(
    Xvalid,yvalid,
    batch_size=batch_size)
 
checkpointer = ModelCheckpoint(filepath = "/dataset_anass/PId_Best.h5", save_best_only = True, verbose = 1)
learning_rate_reduction=ReduceLROnPlateau(monitor='val_accuracy', patience = 3, verbose = 1, factor = 0.5, minlr = 0.00001)
 

start = time.time()
 
# let's get started !
 
history=model.fit_generator(train_generator,
                            epochs=epochs,
                            validation_data = validation_generator,
                            verbose=1,
                            steps_per_epoch=len(Xtrain) // batch_size,
                            validation_steps=len(Xvalid) //batch_size,
                            callbacks=[checkpointer, learning_rate_reduction]
                           )
 
end = time.time()
 
duration = end - start
print ('\n This Model took %0.2f seconds (%0.1f minutes) to train for %d epochs'%(duration, duration/60, epochs) )

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=(150, 150,3), activation='relu', name='Conv2D_1'))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', name='Conv2D_2'))
model.add(MaxPool2D(pool_size=(2,2), name='Maxpool_1'))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name='Conv2D_3'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='Conv2D_4'))
model.add(MaxPool2D(pool_size=(2,2), name='Maxpool_2'))
model.add(Dropout(0.25))
    
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name='Conv2D_5'))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', name='Conv2D_6'))
model.add(MaxPool2D(pool_size=(2,2), name='Maxpool_3'))

model.add(Flatten())
model.add(Dense(units=512, activation='relu', name='Dense_1'))
model.add(Dropout(0.5))
#model.add(Dense(units=128, activation='relu', name='Dense_2'))
model.add(Dense(units=85, activation='softmax', name='Output'))

In [ ]:
# Getting the summary of the model
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers as Optimizer
model.compile( optimizer=Optimizer.Adam(lr=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
 
# Time to train our model !
epochs = 100#100
batch_size=64#32
 
train_datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=True
)
 
test_datagen = ImageDataGenerator()
 
train_generator = train_datagen.flow(
    Xtrain,ytrain,
    batch_size=batch_size)
 
validation_generator = test_datagen.flow(
    Xvalid,yvalid,
    batch_size=batch_size)
 
checkpointer = ModelCheckpoint(filepath = "/dataset_anass/PId_Best.h5", save_best_only = True, verbose = 1)
learning_rate_reduction=ReduceLROnPlateau(monitor='val_accuracy', patience = 3, verbose = 1, factor = 0.5, minlr = 0.00001)
 

start = time.time()
 
# let's get started !
 
history=model.fit_generator(train_generator,
                            epochs=epochs,
                            validation_data = validation_generator,
                            verbose=1,
                            steps_per_epoch=len(Xtrain) // batch_size,
                            validation_steps=len(Xvalid) //batch_size,
                            callbacks=[checkpointer, learning_rate_reduction]
                           )
 
end = time.time()
 
duration = end - start
print ('\n This Model took %0.2f seconds (%0.1f minutes) to train for %d epochs'%(duration, duration/60, epochs) )